In [20]:
#word2vec
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from collections import Counter
import operator
import collections


#import fasttext.util

#utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

#src
#root_project = "/content/ReSt/"
root_project = "/Users/Alessandro/Dev/repos/ReSt/"
#root_project = "/home/jupyter/ReSt/"

sys.path.append(root_project)
from src.data.utils import load_csv_to_dict, dtype, dtype_transformation, set_unkmark_token
from src.data.word_embedding import get_index_key_association, get_int_seq, build_keras_embedding_matrix, get_data_to_emb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dataset

In [2]:
#path
#dataset_path    = "/Users/Alessandro/Dev/repos/ReSt/dataset/haspeede2/preprocessed/reference/reference_tweets.csv"
dataset_path    = root_project + 'dataset/haspeede2/preprocessed/dev/dev.csv'
w2v_bin_path    = root_project + 'results/model/word2vec/twitter128.bin'
#w2v_bin_path    = root_project + 'results/model/word2vec/tweets_2019_Word2Vect.bin'

In [23]:
dataset = load_csv_to_dict(dataset_path)
senteces = dataset["tokens"]

Converting data 'lemma' String in list of String
Converting data 'pos' String in list of String
Converting data 'dep' String in list of String
Converting data 'word_polarity' String in list of String
Converting data 'tokens' String in list of String
Converting data 'stem' String in list of String


In [4]:
senteces[29]

['Italiani',
 '!',
 '!',
 'via',
 'da',
 "l'",
 'Italia',
 'questo',
 'territorio',
 'ormai',
 'è',
 'di',
 'i',
 'profughi',
 '.']

In [5]:
dtype(dataset)

id                   <class 'int'>
text                 <class 'str'>
hs                   <class 'int'>
stereotype           <class 'int'>
processed_text       <class 'str'>
text_length          <class 'int'>
hashtags             <class 'int'>
%CAPS-LOCK words     <class 'int'>
esclamations         <class 'int'>
questions            <class 'int'>
tokens               <class 'list'>
lemma                <class 'list'>
pos                  <class 'list'>
dep                  <class 'list'>
word_polarity        <class 'list'>
sentence_positive    <class 'float'>
sentence_negative    <class 'float'>
sentence_neutral     <class 'float'>
stem                 <class 'list'>
%bad_words           <class 'int'>


In [6]:
sentece_i = 53
print("Examples sentence: {}".format(dataset["text"][sentece_i]))
print("To tokens: {}".format(dataset["tokens"][sentece_i]))

Examples sentence: +++#Siria🇸🇾 Evacuati civili dalla città terroristi #IS avanzano dopo violenti scontri con Esercito #siriano a #Palmira ma non è finita+++ 
To tokens: ['+', '+', '<', 'Siria', '>', 'bandiera', 'siria', 'Evacuati', 'civili', 'da', 'la', 'città', 'terroristi', '<', 'Is', '>', 'avanzano', 'dopo', 'violenti', 'scontri', 'con', 'Esercito', '<', 'siriano', '>', 'a', '<', 'Palmira', '>', 'ma', 'non', 'è', 'finita', '+', '+']


## useful information

In [7]:
#data
n_sentences = len(senteces)
unique_words = set([word for words in senteces for word in words])
unique_words_freq = dict(Counter(i for sub in senteces for i in set(sub)))
n_unique_words = len(unique_words)

#print data
print(" - #sentences: {}".format(n_sentences))
print(" - Unique word on the datset: {}".format(n_unique_words))

 - #sentences: 6839
 - Unique word on the datset: 20594


## W2V

In [8]:
token_setences = dataset["tokens"]

In [9]:
#w2v_model = Word2Vec.load(w2v_bin_path)
wv = KeyedVectors.load_word2vec_format(datapath(w2v_bin_path), binary=True)

In [18]:
wv["africani"]

array([-5.5743259e-01, -1.0227908e+00,  8.0411959e-01, -1.8877728e+00,
       -1.1589552e+00, -2.5145683e-01, -7.0758271e-01, -9.4391364e-01,
       -4.7587892e-01, -2.1063483e+00,  1.3505214e-01, -7.4882346e-01,
        1.1972415e+00, -4.1316342e-02,  1.3334475e-01, -1.0462364e+00,
        4.5099264e-01,  2.5281479e+00,  4.4037056e-01,  1.6277286e+00,
        2.7041452e+00, -1.5561391e+00, -9.4343483e-01, -2.3908161e-01,
        9.4067627e-01,  5.3680116e-01, -2.2288663e+00, -5.9790820e-01,
       -3.4088647e-01, -6.6892159e-01, -1.4420565e+00, -1.6446360e+00,
       -2.1486742e+00,  4.2901948e-01, -6.0259920e-01, -2.3585441e+00,
        8.4915322e-01,  2.4764334e-01,  2.6220727e-01,  7.2281873e-01,
        2.0849390e+00,  4.0762094e-01, -1.5244249e+00,  2.2978106e+00,
       -1.9623476e-01, -8.4033400e-01, -1.5208732e+00, -1.0290638e+00,
        2.3683529e+00,  1.1123921e-01,  3.0179641e-01, -1.2741308e-01,
        2.6779404e+00, -4.0727234e-01, -1.4980392e+00,  2.2269785e+00,
      

In [11]:
#len(w2v_model.wv.vocab.keys())
len(wv.vocab.keys())

1170776

In [28]:
wv.vectors.shape

(1170776, 128)

In [12]:
know_words   = []
unknow_words = []
for word in unique_words:
    if word in wv.vocab.keys():
        know_words.append(word)
    else:
        unknow_words.append(word)
print("know words: {}".format(len(know_words)))
print("unknow words: {}".format(len(unknow_words)))

know words: 19483
unknow words: 1111


In [13]:
unknow_words_freq = {word: unique_words_freq[word] for word in unknow_words}                     
unknow_words_freq_sorted = sorted(unknow_words_freq.items(),key=operator.itemgetter(1),reverse=True)
unknow_words_freq_sorted[:111]

[("po'", 29),
 ('integrar', 14),
 ('webitalia', 7),
 ('chiamarli', 7),
 ('sottometter', 6),
 ('prostituirsi', 5),
 ('accoglierne', 4),
 ('centrisociale', 4),
 ('derubò', 4),
 ('facendoli', 3),
 ('Merdo', 3),
 ('ladruncole', 3),
 ('usarli', 3),
 ('sostituirli', 3),
 ('derubarono', 3),
 ('educarli', 3),
 ('comportar', 3),
 ('araboafricani', 3),
 ('darli', 3),
 ('rapinar', 3),
 ('smaltiva', 3),
 ('Randieri', 3),
 ('cartabianca', 3),
 ('costituirsi', 3),
 ('Afroislamica', 2),
 ('Dioti', 2),
 ('Megalizzi', 2),
 ('vaccinale', 2),
 ('Tornerebbero', 2),
 ('fregandosene', 2),
 ('accoglioni', 2),
 ('sgozzar', 2),
 ('impuni', 2),
 ('Metapolitica', 2),
 ('farne', 2),
 ('scaricarli', 2),
 ('fottiti', 2),
 ('mert', 2),
 ('cercarti', 2),
 ('dimarted', 2),
 ('Tracing', 2),
 ('Mantener', 2),
 ('ricattar', 2),
 ('lasciandoci', 2),
 ('diciotti', 2),
 ('Integrar', 2),
 ('Liberar', 2),
 ('microaree', 2),
 ('identificarli', 2),
 ('riconoscersi', 2),
 ('adeguar', 2),
 ('mandandoli', 2),
 ('Impugnamo', 2),
 (

# build keras embedding matrix

In [45]:
def get_index_key_association(wv):
    key_to_index = {"<UNK>": 0}
    index_to_key = {0: "<UNK>"}
    for idx, word in enumerate(sorted(wv.vocab)):
        key_to_index[word]  = idx+1 # which row in `weights` corresponds to which word?
        index_to_key[idx+1] = word # which row in `weights` corresponds to which word?
    return index_to_key, key_to_index

def build_keras_embedding_matrix(wv, index_to_key=None):
    print('Vocab_size is {}'.format(len(wv.vocab)))
    vec_size = wv.vector_size
    vocab_size = len(wv.vocab) + 1 # plus the unknown word
    
    if index_to_key is None:
        index_to_key, _ = get_index_key_association(wv)
    # Create the embedding matrix where words are indexed alphabetically
    embedding_matrix = np.zeros(shape=(vocab_size, vec_size))
    for idx in index_to_key: 
        #jump the first, words not found in embedding int 0 and will be all-zeros
        if idx != 0:
            embedding_matrix[idx] = wv.get_vector(index_to_key[idx])

    print('Embedding_matrix with unk word loaded')
    print('Shape {}'.format(embedding_matrix.shape))
    return embedding_matrix, vocab_size

In [46]:
index_to_key, key_to_index = get_index_key_association(wv)
embedding_matrix, vocab_size = build_keras_embedding_matrix(wv, index_to_key)

Vocab_size is 1170776
Embedding_matrix loaded
Shape (1170777, 128)
